In [ ]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import RMSprop

In [ ]:
ruta = 'texto.txt'
with open(ruta, 'r') as archivo:
    texto = archivo.read().lower()

tokenizador = RegexpTokenizer(r'\w+')
palabras = tokenizador.tokenize(texto)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(palabras)
total_palabras = len(tokenizer.word_index) + 1

LONGITUD_SECUENCIA = 5
secuencias = []
for i in range(LONGITUD_SECUENCIA, len(palabras)):
    secuencia = palabras[i-LONGITUD_SECUENCIA:i+1]
    secuencias.append(secuencia)

secuencias = tokenizer.texts_to_sequences(secuencias)
secuencias = np.array(secuencias)
X, Y = secuencias[:, :-1], secuencias[:, -1]

X_one_hot = np.zeros((X.shape[0], X.shape[1], total_palabras), dtype=np.bool_)
for i, secuencia in enumerate(X):
    for t, palabra in enumerate(secuencia):
        X_one_hot[i, t, palabra] = 1

Y = np.eye(total_palabras)[Y]

In [ ]:
def predecir_siguiente_palabra(modelo, texto, tokenizer, LONGITUD_SECUENCIA, total_palabras):
    tokens = tokenizer.texts_to_sequences([texto.split()[-LONGITUD_SECUENCIA:]])[0]
    tokens_padded = np.pad(tokens, (LONGITUD_SECUENCIA - len(tokens), 0), mode='constant')
    tokens_one_hot = np.zeros((1, LONGITUD_SECUENCIA, total_palabras), dtype=np.bool_)
    for t, palabra in enumerate(tokens_padded):
        tokens_one_hot[0, t, palabra] = 1
    prediccion = modelo.predict(tokens_one_hot)
    palabra_predicha = tokenizer.index_word[np.argmax(prediccion)]
    return palabra_predicha


In [ ]:
modelo = Sequential([
    LSTM(128),
    Dense(total_palabras, activation='softmax')
])

modelo.compile(optimizer=RMSprop(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
modelo.summary()

modelo.fit(X_one_hot, Y, batch_size=128, epochs=20, validation_split=0.2)


texto_prueba = "el clima parece"
print(f"Siguiente palabra predicha: {predecir_siguiente_palabra(modelo, texto_prueba, tokenizer, LONGITUD_SECUENCIA, total_palabras)}")


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 4.0081 - val_accuracy: 0.0000e+00 - val_loss: 4.0627
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - accuracy: 0.1765 - loss: 3.8593 - val_accuracy: 0.0000e+00 - val_loss: 4.1572
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.0784 - loss: 3.6670 - val_accuracy: 0.0000e+00 - val_loss: 4.6165
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.0784 - loss: 3.3884 - val_accuracy: 0.0769 - val_loss: 4.3766
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.5098 - loss: 3.1406 - val_accuracy: 0.0769 - val_loss: 5.2589
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.2353 - loss: 2.5449 - val_accuracy: 0.0000e+00 - val_loss: 4.8245
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.3725 - loss: 2.3039 - val_accuracy: 0.0000e+00 - val_loss: 4.3358
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.6078 - loss: 1.7743 - val_accuracy: 0.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
Siguiente palabra predicha: despejado
